Short Summary at https://docs.google.com/document/d/1-FUR143YcUYACe57fZ7yAZrDTEbvD8u5XsjVoHu2XQE/edit?usp=sharing

In [0]:
import tensorflow as tf
from tensorflow.keras import backend, datasets, optimizers, regularizers, utils
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import (Conv2D, Activation, Lambda, Add, 
BatchNormalization, Dense, GlobalAveragePooling2D, Dropout)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
%matplotlib inline

In [0]:
backend.set_image_data_format('channels_last')
l=1e-4

In [0]:
class BasicBlock(Model):
  """ Basic Block with identity mapping in shortcuts (Option A)."""
  def __init__(self, out_channels, stride=(1,1)):
    super().__init__()
    self.conv1 = Conv2D(out_channels, 3, strides=stride, padding='same', kernel_regularizer=regularizers.l2(l=l))
    self.bn1 = BatchNormalization()
    self.relu1 = Activation('relu')
    self.conv2 = Conv2D(out_channels, 3, padding='same', kernel_regularizer=regularizers.l2(l=l))
    self.bn2 = BatchNormalization()
    if stride[0] != 1:
      self.shortcut = Conv2D(out_channels, 1, strides=2, padding='same', kernel_regularizer=regularizers.l2(l=l))
    else:
      self.shortcut = Lambda(lambda x: x)
    self.bn3 = BatchNormalization()
    self.add = Add()
    self.relu2 = Activation('relu')

  def call(self, x):
    y = self.conv1(x)
    y = self.bn1(y)
    y = self.relu1(y)
    y = self.conv2(y)
    y = self.bn2(y)
    y = self.add([self.bn3(self.shortcut(x)),y])
    y = self.relu2(y)
    return y

In [0]:
class BottleneckBlock(Model):
  """Bottleneck Block with identity mapping in shortcuts (Option A)."""
  def __init__(self, out_channels, stride=(1,1)):
    super().__init__()
    channel = out_channels//4
    self.conv1 = Conv2D(channel, 1, padding='same', kernel_regularizer=regularizers.l2(l=l))
    self.bn1 = BatchNormalization()
    self.relu1 = Activation('relu')
    self.conv2 = Conv2D(channel, 3, padding='same', kernel_regularizer=regularizers.l2(l=l))
    self.bn2 = BatchNormalization()
    self.relu2 = Activation('relu')
    self.conv3 = Conv2D(out_channels, 1, strides=stride, padding='same', kernel_regularizer=regularizers.l2(l=l))
    self.bn3 = BatchNormalization()
    if stride[0] != 1:
      self.shortcut = Conv2D(out_channels, 1, strides=2, padding='same', kernel_regularizer=regularizers.l2(l=l))
    else:
      self.shortcut = Lambda(lambda x: x)
    self.bn4 = BatchNormalization()
    self.add = Add()
    self.relu3 = Activation('relu')
  
  def call(self, x):
    y = self.conv1(x)
    y = self.bn1(y)
    y = self.relu1(y)
    y = self.conv2(y)
    y = self.bn2(y)
    y = self.relu2(y)
    y = self.conv3(y)
    y = self.bn3(y)
    y = self.add([self.bn4(self.shortcut(x)),y])
    y = self.relu3(y)
    return y

In [0]:
class ResNet(Model):
  def _block(self, channel, n, block, stride=(1,1)):
    layer_block = Sequential()
    layer_block.add(block(channel, stride))
    for i in range(n-1):
      layer_block.add(block(channel))
    return layer_block

  def __init__(self, filters, n, block):
    super().__init__()
    self.conv1 = Conv2D(filters[0], 3, padding='same', input_shape=(32,32,3), kernel_regularizer=regularizers.l2(l=l))
    self.bn1 = BatchNormalization()
    self.relu1 = Activation('relu')
    self.block1 = self._block(filters[0], n, block, stride=(1,1))
    self.block2 = self._block(filters[1], n, block, stride=(2,2))
    self.block3 = self._block(filters[2], n, block, stride=(2,2))
    self.pool = GlobalAveragePooling2D()
    self.fc = Dense(256, activation='relu')
    self.out = Dense(10, activation='softmax')
  
  def call(self, x):
    y = self.conv1(x)
    y = self.bn1(y)
    y = self.relu1(y)
    y = self.block1(y)
    y = self.block2(y)
    y = self.block3(y)
    y = self.pool(y)
    y = self.fc(y)
    y = self.out(y)
    return y

In [0]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
train_images = train_images.astype('float32')
test_labels = test_labels.astype('float32')
train_images, test_images = train_images / 255.0, test_images / 255.0
train_labels = utils.to_categorical(train_labels, 10)
test_labels = utils.to_categorical(test_labels, 10)
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(train_images)
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
print(f'Train Set : {train_images.shape}',f'Test Set : {test_images.shape}')

Train Set : (50000, 32, 32, 3) Test Set : (10000, 32, 32, 3)


In [0]:
model1 = ResNet(filters=[16,32,64], n=5, block=BasicBlock)
model1.build(input_shape=(None, 32, 32, 3))
model1.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])
model1.summary()

Model: "res_net_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_357 (Conv2D)          multiple                  448       
_________________________________________________________________
batch_normalization_489 (Bat multiple                  64        
_________________________________________________________________
activation_339 (Activation)  multiple                  0         
_________________________________________________________________
sequential_27 (Sequential)   multiple                  24160     
_________________________________________________________________
sequential_28 (Sequential)   multiple                  90336     
_________________________________________________________________
sequential_29 (Sequential)   multiple                  356800    
_________________________________________________________________
global_average_pooling2d_9 ( multiple                  0

In [0]:
#model1.fit(train_images, train_labels, batch_size=128, epochs=100, validation_data=(test_images, test_labels))
model1.fit(datagen.flow(train_images, train_labels, batch_size=128), 
           steps_per_epoch=train_images.shape[0]//128, epochs = 100, validation_data=(test_images, test_labels))

Epoch 1/100
390/390 [==============================] - 30s 78ms/step - loss: 1.6732 - accuracy: 0.4267 - val_loss: 1.7973 - val_accuracy: 0.4098
Epoch 2/100
390/390 [==============================] - 30s 76ms/step - loss: 1.2842 - accuracy: 0.5812 - val_loss: 1.7579 - val_accuracy: 0.4964
Epoch 3/100
390/390 [==============================] - 30s 76ms/step - loss: 1.1091 - accuracy: 0.6497 - val_loss: 1.5882 - val_accuracy: 0.5193
Epoch 4/100
390/390 [==============================] - 30s 76ms/step - loss: 0.9909 - accuracy: 0.6931 - val_loss: 1.6361 - val_accuracy: 0.5412
Epoch 5/100
390/390 [==============================] - 30s 77ms/step - loss: 0.8944 - accuracy: 0.7294 - val_loss: 1.3572 - val_accuracy: 0.6319
Epoch 6/100
390/390 [==============================] - 30s 76ms/step - loss: 0.8359 - accuracy: 0.7533 - val_loss: 0.9201 - val_accuracy: 0.7306
Epoch 7/100
390/390 [==============================] - 29s 75ms/step - loss: 0.7836 - accuracy: 0.7728 - val_loss: 0.9746 - val_ac

In [0]:
model2 = ResNet(filters=[16,32,64], n=10, block=BottleneckBlock)
model2.build(input_shape=(None, 32, 32, 3))
model2.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])
model2.summary()

Model: "res_net_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_390 (Conv2D)          multiple                  448       
_________________________________________________________________
batch_normalization_535 (Bat multiple                  64        
_________________________________________________________________
activation_370 (Activation)  multiple                  0         
_________________________________________________________________
sequential_30 (Sequential)   multiple                  4560      
_________________________________________________________________
sequential_31 (Sequential)   multiple                  14976     
_________________________________________________________________
sequential_32 (Sequential)   multiple                  52480     
_________________________________________________________________
global_average_pooling2d_10  multiple                  0

In [0]:
#model2.fit(train_images, train_labels, batch_size=128, epochs=100, validation_data=(test_images, test_labels))
model2.fit(datagen.flow(train_images, train_labels, batch_size=128), 
           steps_per_epoch=train_images.shape[0]//128, epochs = 100, validation_data=(test_images, test_labels))

Epoch 1/100
390/390 [==============================] - 39s 100ms/step - loss: 2.2391 - accuracy: 0.2076 - val_loss: 2.9519 - val_accuracy: 0.1118
Epoch 2/100
390/390 [==============================] - 36s 93ms/step - loss: 2.0463 - accuracy: 0.2900 - val_loss: 2.0557 - val_accuracy: 0.2848
Epoch 3/100
390/390 [==============================] - 36s 92ms/step - loss: 1.8587 - accuracy: 0.3626 - val_loss: 1.9168 - val_accuracy: 0.3502
Epoch 4/100
390/390 [==============================] - 36s 93ms/step - loss: 1.7536 - accuracy: 0.4005 - val_loss: 1.7585 - val_accuracy: 0.3926
Epoch 5/100
390/390 [==============================] - 36s 92ms/step - loss: 1.6780 - accuracy: 0.4300 - val_loss: 1.7788 - val_accuracy: 0.3927
Epoch 6/100
390/390 [==============================] - 36s 93ms/step - loss: 1.6087 - accuracy: 0.4525 - val_loss: 1.6037 - val_accuracy: 0.4532
Epoch 7/100
390/390 [==============================] - 36s 92ms/step - loss: 1.5565 - accuracy: 0.4714 - val_loss: 1.6800 - val_a

KeyboardInterrupt: ignored

In [0]:
l = 5*1e-5
model2.fit(datagen.flow(train_images, train_labels, batch_size=128), 
           steps_per_epoch=train_images.shape[0]//128, epochs = 50, initial_epoch=26, validation_data=(test_images, test_labels))

Epoch 27/50
390/390 [==============================] - 35s 91ms/step - loss: 0.9251 - accuracy: 0.7051 - val_loss: 1.1973 - val_accuracy: 0.6386
Epoch 28/50
390/390 [==============================] - 35s 90ms/step - loss: 0.9066 - accuracy: 0.7107 - val_loss: 1.1325 - val_accuracy: 0.6528
Epoch 29/50
390/390 [==============================] - 35s 90ms/step - loss: 0.8903 - accuracy: 0.7179 - val_loss: 1.2638 - val_accuracy: 0.6264
Epoch 30/50
390/390 [==============================] - 35s 90ms/step - loss: 0.8726 - accuracy: 0.7238 - val_loss: 1.2020 - val_accuracy: 0.6403
Epoch 31/50
390/390 [==============================] - 36s 91ms/step - loss: 0.8586 - accuracy: 0.7301 - val_loss: 1.0575 - val_accuracy: 0.6816
Epoch 32/50
390/390 [==============================] - 35s 90ms/step - loss: 0.8379 - accuracy: 0.7372 - val_loss: 1.0261 - val_accuracy: 0.6836
Epoch 33/50
390/390 [==============================] - 35s 89ms/step - loss: 0.8265 - accuracy: 0.7423 - val_loss: 0.9701 - val_ac

In [0]:
model2.fit(datagen.flow(train_images, train_labels, batch_size=128), 
           steps_per_epoch=train_images.shape[0]//128, epochs = 100, initial_epoch=50, validation_data=(test_images, test_labels))

Epoch 51/100
390/390 [==============================] - 36s 93ms/step - loss: 0.6514 - accuracy: 0.8024 - val_loss: 0.7072 - val_accuracy: 0.7859
Epoch 52/100
390/390 [==============================] - 36s 93ms/step - loss: 0.6451 - accuracy: 0.8042 - val_loss: 0.7288 - val_accuracy: 0.7852
Epoch 53/100
390/390 [==============================] - 36s 93ms/step - loss: 0.6382 - accuracy: 0.8102 - val_loss: 0.7332 - val_accuracy: 0.7794
Epoch 54/100
390/390 [==============================] - 36s 93ms/step - loss: 0.6308 - accuracy: 0.8102 - val_loss: 0.9604 - val_accuracy: 0.7259
Epoch 55/100
390/390 [==============================] - 36s 91ms/step - loss: 0.6298 - accuracy: 0.8111 - val_loss: 0.7061 - val_accuracy: 0.7944
Epoch 56/100
390/390 [==============================] - 36s 92ms/step - loss: 0.6159 - accuracy: 0.8149 - val_loss: 0.7210 - val_accuracy: 0.7853
Epoch 57/100
390/390 [==============================] - 36s 91ms/step - loss: 0.6155 - accuracy: 0.8165 - val_loss: 0.7731 -